In [21]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 17 22:32:01 2018

@author: hassan.naseri
"""

import numpy as np
import pandas as pd
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, execute

import warnings
warnings.filterwarnings("ignore")

# This is the number of bits used to address the dataset
# In our example, the shelf number of a fruit is a 4 bit unsigned integer.
NBITS = 4;

In [22]:
def uint4(int_or_string):
    # Simulate 4-bit unsigned integer by masking 4 bits of numpy.unit8
    unit8_type = np.uint8(int_or_string)
    mask = np.uint8(2**4 - 1)
    return(unit8_type & mask)

fileurl = "https://raw.githubusercontent.com/HassanNaseri/quantum-computing-handson/master/fruits2.csv"
data_frame = pd.read_csv(fileurl)

def black_box_check(circuit, key, q):
    # Black box oracle function to encode a winner entry in a dataset
    # Inverts the phase of 'q' if 'key' and 'q' are matching, otherwise the 'q' remains unchanged.
    # Note: the implementation of the oracle here is only for simulation purposes, and its details may noy have 
    # any ptractical relevance. In practice, the oracle is generated systematically for each problem ecncoding 
    # a function (or dataset) that cannot be inverted (or efficiently searched).
    data_dict = data_frame.set_index('name').T.to_dict('records')[0]
    try:
        winner = uint4(data_dict[key])
    except:
        raise ValueError('Invalid key! There is no ' + key)
    if winner&2 == 0:
        circuit.s(q[0])
    if winner&1 == 0:
        circuit.s(q[1])
    circuit.h(q[1])
    circuit.cx(q[0], q[1])
    circuit.h(q[1])
    if winner&2 == 0:
        circuit.s(q[0])
    if winner&1 == 0:
        circuit.s(q[1])

def reflection_about_average(circuit, q):
    # Reflection about average for amplitude amplification
    circuit.h(q[0])
    circuit.h(q[1])    
    circuit.x(q[0])
    circuit.x(q[1])
    circuit.h(q[1])
    circuit.cx(q[0], q[1])
    circuit.h(q[1])
    circuit.x(q[0])
    circuit.x(q[1])
    circuit.h(q[0])
    circuit.h(q[1])

In [23]:
###############################################################################
# The main body code is here:

# The name of fruit to search for its shelf number: 
# Options: apple, banana, orange, strawberry
fruit_to_find = 'orange'

# Define reqisters and circuit
qreg = QuantumRegister(2)
creg = ClassicalRegister(2)
grover = QuantumCircuit(qreg, creg)

# Initialization of quantum registers
grover.h(qreg[0])
grover.h(qreg[1])

# Grover's amplitude amplification (only one iteration here)
black_box_check(grover, fruit_to_find, qreg)
reflection_about_average(grover, qreg)

# Add Measurement
grover.measure(qreg, creg)

# Execute and collect results
job_sim = execute(grover, "local_qasm_simulator", shots=10)
sim_result = job_sim.result()

# Print result counts
print("simulation: ", sim_result)
print(sim_result.get_counts(grover))

shelf = int(max(sim_result.get_counts(grover)), 2)
print("")
print("I found '{:}'! The shlef number is: {:}".format(fruit_to_find, shelf))



simulation:  COMPLETED
{'10': 10}

I found 'orange'! The shlef number is: 2
